In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/mnt/c/Users/Admin/.gcp/newyorktaxi.json'
bucket = 'newyorktaxi-423104-bucket'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set('spark.jars', "../bin/gcs-connector-hadoop3-2.2.22.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .set("spark.hadoop.google.cloud.auth.service.json.keyfile",credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/05/23 15:28:30 WARN Utils: Your hostname, DESKTOP-RJFPN4L resolves to a loopback address: 127.0.1.1; using 172.24.64.142 instead (on interface eth0)
24/05/23 15:28:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/23 15:28:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df_green = spark.read.parquet(f'gs://{bucket}/pq/green/*/*')
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-29 13:42:10|  2020-01-29 13:48:24|                 N|         1|           7|         193|              3|         0.91|        6.0|  0.0|    0.

In [11]:
df_green.createOrReplaceTempView('green')

In [13]:
spark.sql("""
SELECT count(*) from green""").show()

+--------+
|count(1)|
+--------+
| 2304517|
+--------+



In [15]:
df_yellow = spark.read.parquet(f'gs://{bucket}/pq/yellow/*/*')
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2020-01-08 13:40:30|  2020-01-08 13:55:37|              1|         2.61|         1|                 N|         234|         237|           1|       12.0|  0.0|    0.5|      3.06|         0.0|                  0.3

In [16]:
df_yellow.createOrReplaceTempView('yellow')
spark.sql("""
SELECT count(*) from yellow""").show()

+--------+
|count(1)|
+--------+
|39649199|
+--------+

